# Scraping the Data
In order t

In [1]:
import requests 
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re
import h5py
from selenium import webdriver
import time
import json
import csv

In [3]:
base_dir="https://magic.wizards.com/"
base_url="https://magic.wizards.com/en/content/deck-lists-magic-online-products-game-info"

In [7]:
# create a new chrome page. Needs chromedriver.exe to be in path. I put next to anaconda stuff
driver = webdriver.Chrome()
driver.implicitly_wait(30)
driver.get(base_url)
driver.maximize_window()

In [ ]:
#Uses filter to access only certain types of decks

driver.execute_script("window.scrollTo(0, 200)") 
Filterby=driver.find_element_by_id('event_filter')
Filterby.click()
time.sleep(1)
Modern_button=driver.find_element_by_xpath("//li[contains(text(), 'Modern')]")
Modern_button.click()
time.sleep(1)
Search_button=driver.find_element_by_id('custom-search-submit')
Search_button.click()

# select by visible text
# select.select_by_visible_text('Modern')

In [9]:
#After opening the url above, Selenium clicks the scrolls through all the see more archives
'''
scrolls to bottom of page
clicks da fish link to see more info
compares old window height with new one
if equal, continually checks for 60 seconds
if not continues to scroll to the bottom and clicks the link to see more
timeout= time willing to wait for page to load for each click of the see more button
'''

SCROLL_PAUSE_TIME = 1

timeout = 300

while True:

    # Get scroll height
    ### This is the difference. Moving this *inside* the loop
    ### means that it checks if scrollTo is still scrolling 
    last_height = driver.execute_script("return document.body.scrollHeight")

    # Scroll down to bottom
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    
    # Wait to load page
    time.sleep(SCROLL_PAUSE_TIME)
    
    # touch da fishy
    da_fishy = driver.find_element_by_class_name('see-more-article-listing-section') #See more
    da_fishy.click() #click see more

    # Calculate new scroll height and compare with last scroll height
    new_height = driver.execute_script("return document.body.scrollHeight")
    i=0
    while (new_height == last_height) and (i<timeout) :

        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")

        time.sleep(1)
        i+=1
    if i==timeout:
        break



ElementNotVisibleException: Message: element not interactable
  (Session info: chrome=70.0.3538.77)
  (Driver info: chromedriver=2.43.600210 (68dcf5eebde37173d4027fa8635e332711d2874a),platform=Windows NT 6.1.7601 SP1 x86_64)


In [10]:
#Handoff to BS and find the aricles we want
soup=BeautifulSoup(driver.page_source,"html.parser")
# Opens all links and gets all data
links=[]
for link in soup.find("div",{"class":"articles-listing"}).find_all('a'):
    a=link.get('href')
    links.append(a)

#Filtertype=[i.split("/")[-1] for i in links]
len(links)

8941

In [11]:
# Save links to file
linklist=pd.DataFrame(data={"col1": links})
linklist.to_csv("./linklist.csv", sep=',',index=False)

In [4]:
with open('linklist.csv', 'r') as f:
    reader = csv.reader(f)
    links_ls = list(reader)
del links_ls[0]
links = [item for sublist in links_ls for item in sublist]

In [5]:
# Meat of the scrape
'''
Go to each link and scrape the useful info

'''

Headers=['Play Type','Deck Name', 'Card Type','Card Count', 'Card Name', 'Card Link', \
         'Sideboard Card Count', 'Sideboard Card Name']
deck_data_ls=[]
deck_name_ls=[]

for page in links[8000:]:
    page_url=base_dir+page
    r=requests.get(page_url)
    c=r.content
    page_soup=BeautifulSoup(c,"html.parser")
    Play_Type=page.split("/")[-1]
    
    for deckname in page_soup.find_all("div",{"class":"deck-group"}):
        deck_name=deckname.find("span",{"class":"deck-meta"}).h4.string
        deck_name_ls.append(deck_name)
        card_count_ls=[]
        card_name_ls=[]
        card_type=[]
        card_link_ls=[]
        sideb_count=[]
        sideb_name=[]
            
        for deck in deckname.find("div", {"class":"sorted-by-overview-container"}).find_all("div",{"class":"element"}):
            card_type.append(deck.h5.text)
            card_count=[]
            card_name=[]
            card_link=[]

            for cardc in deck.find_all("span",{"class":"card-count"}):
                card_count.append(cardc.text)

            for cardn in deck.find_all("span",{"class":"card-name"}):
                card_name.append(cardn.text)

                try:
                    card_link.append(deck.find("span",{"class":"card-name"}).a.get('href'))
                except AttributeError:
                    card_link.append(deck.find('NaN'))
                    
            card_name_ls.append(card_name)
            card_count_ls.append(card_count)
            card_link_ls.append(card_link)
        try:    
            for sideb in deckname.find("div", {"class":"sorted-by-sideboard-container"}).find_all("span",{"class":"row"}):
                sideb_count.append(sideb.find("span",{"class":"card-count"}).text)
                sideb_name.append(sideb.find("span",{"class":"card-name"}).text)
        except AttributeError:
            sideb_count.append('NaN')
            sideb_name.append('NaN')

        deck_data=dict(zip(Headers,[Play_Type,deck_name,card_type, card_count_ls,\
                                 card_name_ls, card_link_ls, sideb_count, sideb_name])) 
        deck_data_ls.append(deck_data)

In [6]:
Data_temp=pd.DataFrame(columns=Headers)
Data_set=Data_temp.append(deck_data_ls)
Data_set

,Play Type,Deck Name,Card Type,Card Count,Card Name,Card Link,Sideboard Card Count,Sideboard Card Name
0,legacy-daily-2014-11-01,Julian23 (4-0),"[Creature (31), Sorcery (11), Land (18)]","[[2, 1, 2, 4, 4, 4, 4, 4, 1, 1, 4], [4, 4, 3],...","[[Dryad Arbor, Birchlore Rangers, Craterhoof B...",[[http://gatherer.wizards.com/Pages/Search/Def...,"[1, 3, 3, 2, 1, 1, 3, 1]","[Natural Order, Abrupt Decay, Cabal Therapy, C..."
1,legacy-daily-2014-11-01,Samuel (4-0),"[Creature (26), Instant (4), Artifact (7), Lan...","[[1, 4, 2, 4, 3, 2, 2, 4, 4], [4], [4, 1, 1, 1...","[[Brimaz, King of Oreskos, Flickerwisp, Mirran...",[[http://gatherer.wizards.com/Pages/Search/Def...,"[3, 2, 2, 2, 1, 1, 1, 2, 1]","[Absolute Law, Cataclysm, Enlightened Tutor, E..."
2,legacy-daily-2014-11-01,HaggisMcmutton (3-1),"[Creature (29), Instant (5), Artifact (4), Lan...","[[4, 4, 4, 4, 4, 4, 1, 4], [1, 4], [4], [4, 13...","[[Cosi's Trickster, Cursecatcher, Lord of Atla...",[[http://gatherer.wizards.com/Pages/Search/Def...,"[1, 2, 2, 1, 2, 1, 1, 1, 2, 1, 1]","[Daze, Dismember, Echoing Truth, Grafdigger's ..."
3,legacy-daily-2014-11-01,DaPokpok (3-1),"[Creature (24), Artifact (8), Enchantment (6),...","[[2, 4, 2, 2, 4, 2, 4, 4], [4, 4], [2, 4], [4,...","[[Ballyrush Banneret, Captain of the Watch, Co...",[[http://gatherer.wizards.com/Pages/Search/Def...,"[1, 1, 3, 4, 1, 3, 2]","[Karakas, Oblivion Ring, Holy Light, Kor Firew..."
4,legacy-daily-2014-11-01,Svknoe (3-1),"[Planeswalker (3), Creature (2), Sorcery (7), ...","[[3], [2], [4, 1, 2], [4, 3, 4, 1, 4, 1, 2, 2,...","[[Jace, the Mind Sculptor], [Stoneforge Mystic...",[[http://gatherer.wizards.com/Pages/Search/Def...,"[1, 2, 3, 1, 1, 2, 1, 1, 3]","[Engineered Explosives, Pyroblast, Flusterstor..."
5,legacy-daily-2014-11-01,w0rdjuiceBox (3-1),"[Sorcery (23), Instant (13), Artifact (9), Lan...","[[3, 4, 4, 4, 1, 4, 2, 1], [1, 4, 4, 4], [4, 4...","[[Cabal Therapy, Duress, Gitaxian Probe, Infer...",[[http://gatherer.wizards.com/Pages/Search/Def...,"[1, 3, 2, 2, 1, 2, 1, 3]","[Tendrils of Agony, Abrupt Decay, Carpet of Fl..."
6,legacy-daily-2014-11-01,JPA93 (3-1),"[Creature (8), Sorcery (11), Instant (14), Art...","[[4, 4], [3, 4, 4], [4, 2, 4, 4], [4], [4], [3...","[[Emrakul, the Aeons Torn, Griselbrand], [Gita...",[[http://gatherer.wizards.com/Pages/Search/Def...,"[2, 2, 2, 2, 3, 2, 2]","[Blood Moon, Defense Grid, Echoing Truth, Graf..."
7,legacy-daily-2014-11-01,NaLGaS (3-1),"[Creature (10), Sorcery (9), Instant (22), Enc...","[[4, 3, 3], [2, 4, 3], [4, 4, 4, 4, 3, 3], [1]...","[[Delver of Secrets, Tarmogoyf, Young Pyromanc...",[[http://gatherer.wizards.com/Pages/Search/Def...,"[2, 2, 1, 3, 3, 2, 1, 1]","[Electrickery, Flusterstorm, Hydroblast, Krosa..."
8,legacy-daily-2014-11-01,max_i (3-1),"[Planeswalker (3), Creature (3), Sorcery (10),...","[[3], [3], [1, 2, 3, 4], [4, 2, 4, 3, 3], [4],...","[[Jace, the Mind Sculptor], [Snapcaster Mage],...",[[http://gatherer.wizards.com/Pages/Search/Def...,"[1, 1, 2, 1, 1, 1, 1, 1, 2, 2, 1, 1]","[Council's Judgment, Pyroblast, Blue Elemental..."
9,legacy-daily-2014-11-01,sam51087 (3-1),"[Creature (7), Sorcery (8), Instant (22), Arti...","[[4, 3], [1, 4, 3], [4, 3, 3, 4, 1, 3, 4], [2]...","[[Stoneforge Mystic, True-Name Nemesis], [Coun...",[[http://gatherer.wizards.com/Pages/Search/Def...,"[3, 1, 1, 2, 2, 2, 3, 1]","[Flusterstorm, Grafdigger's Cage, Hydroblast, ..."


In [7]:
a=Data_set.to_json(orient='index')

In [8]:
with open('Data_set_Modern_9.json', 'w') as outfile:
    json.dump(a,outfile,indent=4, sort_keys=True, separators=(',', ': '), ensure_ascii=False)

In [3]:
## Data Directory GUI
import sys, os # python modules for interacting with computer
from tkinter import filedialog # interface for opening a file explorer
from tkinter import * # * imports all files in tkinter 
import glob # helps find path 
'''Getting data directory so data can easily be imported '''

root = Tk() # Tk() is a function in tkinter that opens a window
root.directory = filedialog.askdirectory() # opens explorer window so you can find the folder of choice
root.withdraw() # closes the tkinter window since it's unnecessary
oldcwd = os.getcwd() # saves old called working directory (place where data is drawn from) as oldcwd use os.chdir(oldcwd) to go back
os.chdir(root.directory) # sets new directory
newcwd = os.getcwd() # saves new directory name as newcwd

In [7]:
## Load Data use for .csv data
''' Makes a list of all the files with the extension use * to '''

path = root.directory + '/*.json' # Change '' as needed
file_list=glob.glob(path)
file_list

['C:/Users/kbs1/Machine-Learning/Web-Crawler\\Data_set_Modern_1.json',
 'C:/Users/kbs1/Machine-Learning/Web-Crawler\\Data_set_Modern_2.json',
 'C:/Users/kbs1/Machine-Learning/Web-Crawler\\Data_set_Modern_3.json',
 'C:/Users/kbs1/Machine-Learning/Web-Crawler\\Data_set_Modern_4.json',
 'C:/Users/kbs1/Machine-Learning/Web-Crawler\\Data_set_Modern_5.json',
 'C:/Users/kbs1/Machine-Learning/Web-Crawler\\Data_set_Modern_6.json',
 'C:/Users/kbs1/Machine-Learning/Web-Crawler\\Data_set_Modern_7.json',
 'C:/Users/kbs1/Machine-Learning/Web-Crawler\\Data_set_Modern_8.json',
 'C:/Users/kbs1/Machine-Learning/Web-Crawler\\Data_set_Modern_9.json']

In [27]:
## Store all .json in 1 file
#Data_json = pd.DataFrame(index=['Card Count', 'Card Link', 'Card Name', 'Deck Name', 'Play Type', 'Sideboard Card Count','Sideboard Card Name'])
a=0
for file in file_list:
    with open(file, 'r') as f:
        data = json.load(f)
    if a<1:
        Data_json=pd.read_json(data).T
        print(a)
    else:
        print('else')
        print(Data_json.size)
        df1 = pd.read_json(data).T
        Data_json=Data_json.append(df1, ignore_index=True)
        
    a+=1

0
else
160504
else
226184
else
307648
else
378600
else
462176
else
530232
else
670984
else
838480


In [35]:
b=Data_json[['Card Name', 'Card Count']]

In [29]:
a=Data_json.to_json(orient='index')

In [30]:
with open('Data_set_All.json', 'w') as outfile:
    json.dump(a,outfile,indent=4, sort_keys=True, separators=(',', ': '), ensure_ascii=False)

In [37]:
a=b.to_json(orient='index')
with open('Data_set_All_Count_Name.json', 'w') as outfile:
    json.dump(a,outfile,indent=4, sort_keys=True, separators=(',', ': '), ensure_ascii=False)

In [56]:
c=Data_json[Data_json['Play Type'].str.contains('modern')]

In [57]:
d=c[c['Play Type'].str.contains('competitive')]

In [59]:
a=d.to_json(orient='index')
with open('Data_set_Competitive_Modern_All.json', 'w') as outfile:
    json.dump(a,outfile,indent=4, sort_keys=True, separators=(',', ': '), ensure_ascii=False)